In [ ]:
library(Seurat)
library(openxlsx)
library(readxl)
library(Matrix)
library(readr)


Loading required package: SeuratObject

Loading required package: sp

‘SeuratObject’ was built under R 4.3.2 but the current version is
4.3.3; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed

‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.5; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect




In [ ]:
#Need to find out if all duplicates are nonzeros
#Haber
setwd("/dh-projects/ag-conrad/analysis/saal11/panfib/Renaming/Haber")
#Upload matrices
data_haber_matrix <- readMM("data_haber_matrix.mtx")
#Upload original gene list
data_haber_genes <- read.xlsx("data_haber_genes.xlsx")
data_haber_genes <- data_haber_genes$genes
data_haber_cellIDs <- read.delim("data_haber_barcodes.tsv")
rownames(data_haber_matrix) <- data_haber_genes
colnames(data_haber_matrix) <- data_haber_cellIDs$cellIDs

In [19]:
#Upload genes of interest (duplicates)
data_haber_genes_dup <- read.xlsx("data_haber_geneIDs_duplicates_forsum.xlsx")

In [20]:
#Find the zeros of these duplicates
#Initialize an empty list to store the sums
sums_list <- list()

#Iterate over each gene in your list
for (gene in data_haber_genes_dup$Old) {
  #Subset the count matrix for the current gene and calculate the sum
  gene_sum <- sum(data_haber_matrix[rownames(data_haber_matrix) == gene, ])
  
  #Add the sum to the sums_list with the gene name as the key
  sums_list[[gene]] <- gene_sum
}

In [ ]:
sums_list

In [3]:
#Intermediate box
setwd("/dh-projects/ag-conrad/analysis/saal11/panfib/Renaming/Haber")
data_haber_matrix <- readMM("data_haber_matrix.mtx")
data_haber_cellIDs <- read.delim("data_haber_barcodes.tsv")
colnames(data_haber_matrix) <- data_haber_cellIDs$cellIDs
data_haber_genes_manual <- read.xlsx("data_haber_geneIDs_int_removed_manualedit.xlsx")
rownames(data_haber_matrix) <- data_haber_genes_manual$New

In [11]:
#Duplicates
duplicate_genes <- duplicated(rownames(data_haber_matrix))
cat("Duplicate gene IDs:\n")
print(rownames(data_haber_matrix)[duplicate_genes])
duplicated <- rownames(data_haber_matrix)[duplicate_genes]

Duplicate gene IDs:
 [1] "BTBD8"        "HNRNPU"       "RRM2"         "LINC01291"    "ROPN1L"      
 [6] "GCNT4"        "TRAF3IP2-AS1" "LINC00632"    "DLGAP2"       "UBE2R2-AS1"  
[11] "TMEM8B"       "S1PR3"        "TTLL11"       "MICAL2"       "NEBL"        
[16] "C10orf67"     "CC2D2B"       "LCOR"         "CCND2-AS1"    "KRT6A"       
[21] "PLEKHG7"      "SUCLA2"       "MIA2"         "MEG8"         "MEG8"        
[26] "ZFHX3"        "MYO18A"       "SOCS4"        "SEPTIN4"      "IGFLR1"      
[31] "KLKB1"        "SIGLEC12"    


In [12]:
duplicated

[1] "BTBD8"        "HNRNPU"       "RRM2"         "LINC01291"    "ROPN1L"      
 [6] "GCNT4"        "TRAF3IP2-AS1" "LINC00632"    "DLGAP2"       "UBE2R2-AS1"  
[11] "TMEM8B"       "S1PR3"        "TTLL11"       "MICAL2"       "NEBL"        
[16] "C10orf67"     "CC2D2B"       "LCOR"         "CCND2-AS1"    "KRT6A"       
[21] "PLEKHG7"      "SUCLA2"       "MIA2"         "MEG8"         "MEG8"        
[26] "ZFHX3"        "MYO18A"       "SOCS4"        "SEPTIN4"      "IGFLR1"      
[31] "KLKB1"        "SIGLEC12"

In [13]:
subset_matrix_dup <- data_haber_matrix[rownames(data_haber_matrix) %in% duplicated, ]

In [14]:
genes_to_keep <- setdiff(rownames(data_haber_matrix), duplicated)
subset_matrix_main <- data_haber_matrix[rownames(data_haber_matrix) %in% genes_to_keep, ]

In [63]:
str(subset_matrix_dup)

 num [1:63, 1:114396] 0 0 0 1 0 0 0 0 0 0 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:63] "BTBD8" "BTBD8" "HNRNPU" "HNRNPU" ...
  ..$ : chr [1:114396] "F00431_AGAGCTTGTAGAGGAA" "F01380_GATCTAGTCTTGCCGT" "F01391_AGTCTTTAGGTTACCT" "F01391_ATCCACCAGGAGTCTG" ...


In [65]:
str(subset_matrix_main)

 num [1:26314, 1:114396] 0 0 0 0 0 0 0 0 0 0 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:26314] "RP11-34P13.3" "RP11-34P13.7" "RP11-34P13.14" "FO538757.3" ...
  ..$ : chr [1:114396] "F00431_AGAGCTTGTAGAGGAA" "F01380_GATCTAGTCTTGCCGT" "F01391_AGTCTTTAGGTTACCT" "F01391_ATCCACCAGGAGTCTG" ...


In [15]:
#Aggregate using Matrix package
subset_matrix_dup <- as(subset_matrix_dup, "dgCMatrix")
gene_names <- rownames(subset_matrix_dup)
unique_gene_names <- unique(gene_names)
gene_factors <- factor(gene_names, levels = unique_gene_names)
aggregated_counts <- rowsum(as.matrix(subset_matrix_dup), group = gene_factors)

In [16]:
aggregated_counts

,F00431_AGAGCTTGTAGAGGAA,F01380_GATCTAGTCTTGCCGT,F01391_AGTCTTTAGGTTACCT,F01391_ATCCACCAGGAGTCTG,F01391_CTCGTCATCATCTGCC,F01391_CTGATAGCAGACGCCT,F01391_CTGGTCTCACAGAGGT,F01391_GAACATCAGCTGATAA,F01391_TACTCATCATAGAAAC,F01391_TCAGGATTCGGTGTTA,⋯,ILD62-1_AACTGGTAGAAACCAT,ILD62-1_AGTAGTCGTGCAGGTA,ILD62-1_CATTATCGTAGCAAAT,ILD62-1_TCAGCTCGTAGAAAGG,ILD62-1_TTAGTTCCAGCTCGAC,ILD62-2_ACGAGGATCTGCCAGG,ILD63_AACTCTTCAGACTCGC,ILD63_CAGAATCTCCACGTTC,ILD63_CTGAAACAGAAGGTGA,ILD63_GGCCGATAGCTGTTCA
BTBD8,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
HNRNPU,1,1,1,1,1,1,1,0,0,1,⋯,1,0,6,1,1,0,3,0,0,0
RRM2,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
LINC01291,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
KLKB1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ROPN1L,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
GCNT4,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
TRAF3IP2-AS1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
LINC00632,0,0,0,0,0,0,0,0,0,0,⋯,1,0,3,0,0,0,0,0,0,0
DLGAP2,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [68]:
str(aggregated_counts)

 num [1:31, 1:114396] 0 1 0 0 0 0 0 0 0 0 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:31] "BTBD8" "HNRNPU" "RRM2" "LINC01291" ...
  ..$ : chr [1:114396] "F00431_AGAGCTTGTAGAGGAA" "F01380_GATCTAGTCTTGCCGT" "F01391_AGTCTTTAGGTTACCT" "F01391_ATCCACCAGGAGTCTG" ...


In [17]:
combined_matrix <- rbind(subset_matrix_main, aggregated_counts)

In [20]:
# Initialize an empty list to store the sums
sums_list <- list()

# Iterate over each gene in your list
for (gene in duplicated) {
  # Subset the count matrix for the current gene and calculate the sum
  gene_sum <- sum(combined_matrix[rownames(combined_matrix) == gene, ])
  
  # Add the sum to the sums_list with the gene name as the key
  sums_list[[gene]] <- gene_sum
}

In [21]:
sums_list

$BTBD8
[1] 3399

$HNRNPU
[1] 166463

$RRM2
[1] 8302

$LINC01291
[1] 187

$ROPN1L
[1] 69841

$GCNT4
[1] 1854

$`TRAF3IP2-AS1`
[1] 4557

$LINC00632
[1] 1887

$DLGAP2
[1] 379

$`UBE2R2-AS1`
[1] 49

$TMEM8B
[1] 4709

$S1PR3
[1] 2540

$TTLL11
[1] 1928

$MICAL2
[1] 16270

$NEBL
[1] 15638

$C10orf67
[1] 7785

$CC2D2B
[1] 170

$LCOR
[1] 19391

$`CCND2-AS1`
[1] 276

$KRT6A
[1] 4609

$PLEKHG7
[1] 8103

$SUCLA2
[1] 16494

$MIA2
[1] 21652

$MEG8
[1] 560

$ZFHX3
[1] 26466

$MYO18A
[1] 5057

$SOCS4
[1] 17796

$SEPTIN4
[1] 5426

$IGFLR1
[1] 30365

$KLKB1
[1] 386

$SIGLEC12
[1] 2107

In [18]:
head(combined_matrix)

  [[ suppressing 34 column names ‘F00431_AGAGCTTGTAGAGGAA’, ‘F01380_GATCTAGTCTTGCCGT’, ‘F01391_AGTCTTTAGGTTACCT’ ... ]]



6 x 114396 sparse Matrix of class "dgCMatrix"
                                                                               
RP11-34P13.3  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
RP11-34P13.7  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
RP11-34P13.14 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
FO538757.3    . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
FO538757.2    . . . . . . . . 1 . . . . 1 . . 1 . 1 . . . . 1 2 . . . . . 1 . .
AP006222.2    . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
                      
RP11-34P13.3  . ......
RP11-34P13.7  . ......
RP11-34P13.14 . ......
FO538757.3    . ......
FO538757.2    1 ......
AP006222.2    . ......

 .....suppressing 114362 columns in show(); maybe adjust options(max.print=, width=)
 ..............................

In [70]:
str(combined_matrix)

 num [1:26345, 1:114396] 0 0 0 0 0 0 0 0 0 0 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:26345] "RP11-34P13.3" "RP11-34P13.7" "RP11-34P13.14" "FO538757.3" ...
  ..$ : chr [1:114396] "F00431_AGAGCTTGTAGAGGAA" "F01380_GATCTAGTCTTGCCGT" "F01391_AGTCTTTAGGTTACCT" "F01391_ATCCACCAGGAGTCTG" ...


In [71]:
#Load metadata and dim reductions
data_haber_metadata <- read_csv("data_haber_metadata_final.csv")

New names:
• `` -> `...1`
• `...1` -> `...2`
Rows: 114396 Columns: 18
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (13): ...2, subclass, cells, lineage, condition, aetiology, study, tissu...
dbl  (5): ...1, nCount_RNA, nFeature_RNA, percent.mt, age

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [72]:
data_haber_metadata <- as.data.frame(data_haber_metadata)

In [74]:
rownames(data_haber_metadata) <- data_haber_metadata$`...2`

In [75]:
data_haber_metadata$`...2` <- NULL
data_haber_metadata$`...1` <- NULL

In [77]:
#Create Seurat
data_haber_seurat <- CreateSeuratObject(counts = combined_matrix, meta.data = data_haber_metadata)

Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”


In [78]:
data_haber_seurat

An object of class Seurat 
26345 features across 114396 samples within 1 assay 
Active assay: RNA (26345 features, 0 variable features)
 1 layer present: counts

In [79]:
setwd("/dh-projects/ag-conrad/analysis/saal11/panfib/Renaming/Haber")
save(data_haber_seurat, file = "data_haber_seurat_final.RData")

In [40]:
#Sparsity

total_elements <- length(data_haber_matrix)
# Calculate the number of zero elements in the matrix
zero_elements <- sum(data_haber_matrix == 0)

# Calculate the sparsity
sparsity <- zero_elements / total_elements

In [41]:
sparsity

[1] 0.9192754